In [2]:
import os
import sys
from pathlib import Path

cwd = Path(os.getcwd())
if cwd.name != "backend":
    repo = cwd.parent.parent
    sys.path.append(str(cwd))
    sys.path.append(str(repo))
    
    os.chdir(repo / "src" / "backend")

from data.evaluation.src.jupyter_utils.preamble import *
import ocel_generator as ocelgen
from datetime import datetime, timedelta
from importlib import reload

# Pallet Delivery Process

In [ ]:
reload(ocelgen)

otypes = ["Order", "Item", "Pallet", "Vehicle"]
activities = ["receive order", "get item", "load", "deliver", "inspect"]

# rng = np.random.default_rng(seed=42)

gen = ocelgen.OcelGenerator(
    otypes=otypes,
    activities=activities,
    oid_prefixes={"Order": "o", "Item": "i", "Pallet": "p", "Vehicle": "v"},
)

orders = [gen.obj(otype="Order") for _ in range(2)]
items = [gen.obj(otype="Item") for _ in range(4)]
pallets = [gen.obj(otype="Pallet") for _ in range(3)]
vehicles = [gen.obj(otype="Vehicle", attr=dict(consumption=c)) for c in [.10, .125]]

o1, o2 = orders
i1, i2, i3, i4 = items
p1, p2, p3 = pallets
v1, v2 = vehicles

gen.ev("receive order", eid="receive_o1", timestamp=datetime(2024, 5, 7, 21, 0, 0), objs=[o1])
e = gen.ev("get item", eid="get_i1", timestamp=datetime(2024, 5, 10, 18, 0, 0), objs=[o1, i1])
gen.ev("receive order", eid="receive_o2", timestamp=datetime(2024, 5, 14, 10, 30, 0), objs=[o2])
e = gen.ev("load", eid="load_p1", dt=timedelta(minutes=90), objs=[i1, p1, v1])
e = gen.ev("deliver", eid="deliver_1", dt=timedelta(hours=1), objs=[p1, v1], attr=dict(distance=52))

e = gen.ev("get item", eid="get_i3", dt=timedelta(hours=24), objs=[o2, i3])
e = gen.ev("get item", eid="get_i2", dt=timedelta(days=5), objs=[o1, i2])
e = gen.ev("get item", eid="get_i4", dt=timedelta(minutes=10), objs=[o2, i4])

e = gen.ev("load", eid="load_p2", dt=timedelta(minutes=50), objs=[i2, p2, v2])
e = gen.ev("load", eid="load_p3", dt=timedelta(minutes=5), objs=[i3, i4, p3, v2])

e = gen.ev("deliver", eid="deliver_2a", dt=timedelta(minutes=55), objs=[p2, p3, v2], attr=dict(distance=20))
e = gen.ev("deliver", eid="deliver_2b", dt=timedelta(minutes=42), objs=[p3, v2], attr=dict(distance=40))

e = gen.ev("inspect", eid="inspect_v1", dt=timedelta(hours=47, minutes=18), objs=[v1], attr=dict(distance=118))
e = gen.ev("inspect", eid="inspect_v2", timestamp=datetime(2024, 5, 26, 16, 30, 0), objs=[v2], attr=dict(distance=115))

ocel = gen.generate()
# table = ocel.ocel.get_extended_table()
# attrs = ocel.eattr_names
# tbl_otypes = [col[10:] for col in table.columns[-len(ocel.otypes):]]
# table.columns = ["event", "activity", "timestamp", *attrs, *tbl_otypes]

table = ocel.events.join(
    (
        ocel.relations.groupby(["ocel:eid", "ocel:type"])["ocel:oid"]
        .agg(sorted)
        .apply(", ".join)
        .unstack()
        .sort_index(key=pd_util.index_order(otypes), axis=1)
    ),
    on="ocel:eid",
).rename(
    columns={
        "ocel:eid": "Event",
        "ocel:activity": "Activity",
        "ocel:timestamp": "Timestamp",
        **{attr: attr.capitalize() for attr in ocel.eattr_names},
        **{ot: ot.capitalize() for ot in ocel.otypes},
    }
)
table

In [ ]:
from ocel.default_ocel import OCEL_BASE_PATH

pm4py.write_ocel2_sqlite(ocel.ocel, str(OCEL_BASE_PATH / "pallet-logistics-v1.sqlite"))

In [ ]:
ttformatter = lambda x: re.sub(r"([\w\\ ]+)", "\\\\texttt{\\1}", x)

ocel.relations["tup"] = ocel.relations.apply(
    lambda row: f'({row["ocel:eid"]}, {row["ocel:oid"]})', axis=1
)
e2o_groups = ocel.relations.groupby("ocel:eid")["tup"].agg(", ".join)

print("-----\nEvents\n")
pd_style.df_to_latex(
    ocel.events[["ocel:eid", "ocel:activity", *ocel.eattr_names, "ocel:timestamp"]],
    label=f"tab:rex-ocel",
    caption="TODO",
    formatters={
        "ocel:timestamp": "{:%m-%d %H:%M}",
        "ocel:eid": ttformatter,
        "ocel:oid": ttformatter,
        "tup": ttformatter,
    },
    col_renamer={
        "space1": "",
        "space2": "",
        "ocel:oid": "Object",
        "ocel:type": "Obj. type",
        "ocel:eid": "Event",
        "ocel:activity": "Activity",
        "ocel:timestamp": "Timestamp",
        "tup": "$\\EtoO$",
        **{attr: attr.capitalize() for attr in ocel.eattr_names},
    },
    escape_columns=False,
    hide_index=True,
)
print("-----\nObjects\n")
objects = ocel.objects[["ocel:oid", "ocel:type", *ocel.oattr_names]].copy()
objects.insert(3, "--", "")
pd_style.df_to_latex(
    objects,
    label=f"tab:rex-ocel",
    caption="TODO",
    formatters={
        "ocel:oid": ttformatter,
    },
    col_renamer={
        "ocel:oid": "Object",
        "ocel:type": "Obj. type",
        **{attr: attr.capitalize() for attr in ocel.oattr_names},
    },
    escape_columns=False,
    hide_index=True,
)
print("-----\nE2O\n")
pd_style.df_to_latex(
    ocel.relations[["ocel:eid", "ocel:oid"]],
    label=f"tab:rex-ocel",
    caption="TODO",
    formatters={
        "ocel:eid": ttformatter,
        "ocel:oid": ttformatter,
    },
    col_renamer={
        "ocel:eid": "Event",
        "ocel:oid": "Object",
    },
    escape_columns=False,
    hide_index=True,
)

In [ ]:
len(ocel.events), len(ocel.objects), len(ocel.relations)

In [7]:
E2OG = gv.Graph()


In [ ]:
alloc = allocation.Allocator.dummy(
    ocel,
    target_otypes={"Order"},
    hu_otypes={"Order", "Item", "Pallet"},
    resource_otypes={"Vehicle"},
    events=1,
    rule=lambda alloc: ar.ClosestTargetsAllocation(
        alloc,
        max_distance=10,
        graph_mode=ag.GraphMode.OBJ_OBJ,
        remove_otype_loops=False,
    )
)
graph_util.nx_to_graphviz(alloc.rule.OG)

In [ ]:
import emissions.object_type_graphs as otgs

otgs.otfg(
    ocel=ocel,
    alloc=alloc,
    graph_mode=ag.GraphMode.OBJ_OBJ,
)